# Text Extraction

NOTES

- The primary key found was endpoint and controller
- The number of requests and responses mismatched. The removed user was chosen was based on the request with the same primary key and only one response. The specific user removed is based on their 0% new year resolution, and as such would likely not have a need for the data visualizations.

## Imports

In [197]:
import re
import json
import pandas as pd

## Data Extraction

### All Data

In [199]:
items = []

# open file
with open("data_cleaned.txt") as f:
    lines = [line.rstrip() for line in f] # read all files and remove whitespace
    for line in lines:
        if re.search("Request|Response", line): # Each request/response makes a new dict
            items.append({})
        elif len(line) == 0: 
            pass # skip if there is no content in the line
        else:
            k, v = re.split(":\s", line) # add the key value pair to the list
            items[-1][k.strip()] = v.strip() # Add to dictionary and remove surrounding whitespace

### Extract Discipline

In [200]:
for i in items:
    i['Discipline'] = re.search("edge\/(.*)$", i['Endpoint']).group(1) # Extract the displine from endpoint using regex

### Match Request and Responses

In [201]:
users = [] # list of users
found = set() # set of found users using Primary Key of discipline and Controller
for i in range(len(items)):
    i1 = items[i]
    if (i1['Controller'], i1['Discipline']) in found: # if the user already exists, skip
        continue
    for j in range(i+1, len(items)):
        i2 = items[j]
        if i1['Controller'] == i2['Controller'] and i1['Discipline'] == i2['Discipline']: # if the primary keys aforementioned match
            users.append({}) # make a new user
            for k, v in i1.items():
                users[-1][k] = v # add all keys and values to user
            for k, v in i2.items():
                users[-1][k] = v # add all keys and values to user
            found.add((i1['Controller'], i1['Discipline']))
            break

### Value Checks

In [202]:
print(
    len(users), users[0]
)

15 {'Address': '178373', 'Red Meat': '100 pounds', 'Controller': 'LocationController', 'Grains': '2 pounds', 'Action': 'Get', 'Endpoint': '(OWL) https://uwo.ca/edge/software', 'Dairy': '200 pounds', 'Cellphone': '55 hours', 'TV': '43 hours', 'Computer': '130 hours', 'New Year Resolution': '5%', 'Discipline': 'software', 'Username': 'Sammy', 'Car': '150 hours', 'Walking': '3 hours', 'Public Transport': '10 hours', 'Status': '4', 'Items/Total': '2/1'}


### Add Conversion Metrics

In [203]:
conversions = {} 
with open('conversion_table.json') as f:
    conversions = json.loads(f.read()) # Using the data provided in the product requirements
conversions

{'Public Transport': 4.3,
 'Car': 6.5,
 'Walking': 0,
 'Red Meat': 8.0,
 'Dairy': 6.3,
 'Grains': 3.7,
 'Cellphone': 3.6,
 'Computer': 4.2,
 'TV': 6.8}

In [204]:
for u in users:
    for k in ['Red Meat', 'Grains', 'Dairy', 'Cellphone', 'TV', 'Computer', 'Car', 'Walking', 'Public Transport']: # List of conversion factors from the text file
        amount, units = re.split("\s", u[k])
        CO2 = int(amount) * conversions[k] # mutliply units by conversion factors
        col_name = f"{k} CO2 Emissions"
        u[col_name] = CO2 # Add new column to table
        u[f"{k} {units}"] = int(amount)
        del u[k]

## Extract CSV

### JSON To CSV Conversion

In [205]:
df = pd.DataFrame(users)
df.head()

,Address,Controller,Action,Endpoint,New Year Resolution,Discipline,Username,Status,Items/Total,Red Meat CO2 Emissions,...,TV hours,Computer CO2 Emissions,Computer hours,Car CO2 Emissions,Car hours,Walking CO2 Emissions,Walking hours,Public Transport CO2 Emissions,Public Transport hours,TV hour
0,178373,LocationController,Get,(OWL) https://uwo.ca/edge/software,5%,software,Sammy,4,2/1,800.0,...,43.0,546.0,130,975.0,150,0,3,43.0,10,NaN
1,156354,Controlmite,Get,(OWL) https://uwo.ca/edge/software,15%,software,Griffin,0,3/1,712.0,...,75.0,369.6,88,487.5,75,0,68,184.9,43,NaN
2,919323,ControlPublic,Get,https://uwo.ca/edge/electrical,9%,electrical,Spencer,0,3/5,208.0,...,50.0,315.0,75,546.0,84,0,65,335.4,78,NaN
3,983575,Controlz,Get,(OWL) https://uwo.ca/edge/mechanical,4%,mechanical,Annie,0,3/1,280.0,...,50.0,21.0,5,130.0,20,0,71,146.2,34,NaN
4,919325,LocationController,Get,https://uwo.ca/edge/mechanical,6%,mechanical,Jessica,0,3/3,440.0,...,100.0,210.0,50,507.0,78,0,101,94.6,22,NaN


### Rename Columns

In [206]:
new_columns = []
for col in df.columns:
    new_columns.append(col.lower().replace(" ", "_")) # Make all columns standardized formats
df.columns = new_columns

### Extract to CSV

In [207]:
df.to_csv("emissions.csv", index=False)